## Setup

In [1]:
# Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from datetime import timedelta
import random

In [2]:
# Cambiamos el directorio para importar las funciones de func_aux
import sys
import os
from pathlib import Path
sys.path.append(str(Path(os.getcwd()).absolute().parent))

In [3]:
from src.funcs_aux import features_rolling_mean_std, optimizacion_precios, crear_price_grid, walk_forward_forecast, features_rolling_template

In [ ]:
datos_unidos = pd.read_csv("../data/procesados/datos_unidos.csv")

In [ ]:
datos_unidos.columns

In [ ]:
cols_categoricas = ['SKU', 'STORE_ID', 'REGION',
       'CITY', 'STATE', 'STORE_TYPE',  'CATEGORY', 'GROUP', 'SUBGROUP', 'GROUP_TYPE',
       'PRICE_GROUP_ID', 'BRAND', "DAY_OF_WEEK"]

target = "TOTAL_SALES"

In [ ]:
for col in cols_categoricas:
    datos_unidos[col] = datos_unidos[col].astype("category")

In [ ]:
datos_unidos.sort_values(by=["DATE", "STORE_ID", "SKU"], inplace=True)

## Feature aggregation

Primero, haremos el promedio de ventas por SKU X STORE_ID de los últimos 7, 30 y 90 días, para compensar el hecho de que no es posible (por limitaciones computacionales) completar todo el dataset con los días en que no hubo transacciones de un producto.

De esta manera, el modelo podrá dilucidar las épocas en donde no hay ventas de ciertos productos.

In [ ]:
datos_unidos = datos_unidos.sort_values("DATE").reset_index(drop=True)
# Guardamos el índice donde cambia la fecha para acceso rápido
cambios_dia = datos_unidos["DATE"].ne(datos_unidos["DATE"].shift()).to_numpy().nonzero()[0]
fechas_unicas = datos_unidos["DATE"].unique()

# Lista única de combinaciones SKU-STORE_ID
combinaciones = datos_unidos[["SKU", "STORE_ID"]].drop_duplicates()

In [ ]:
def rellenar_faltantes(df, fecha):
    # Todas las combinaciones para esta fecha
    comb_fecha = combinaciones.copy()
    comb_fecha["DATE"] = fecha
    # Merge para meter TOTAL_SALES=0 donde falta
    df_completo = comb_fecha.merge(df, on=["SKU", "STORE_ID", "DATE"], how="left")
    df_completo["TOTAL_SALES"] = df_completo["TOTAL_SALES"].fillna(0)
    return df_completo

In [ ]:
buffer = pd.DataFrame()
resultados = []
windows = [7, 30, 90]

for window in windows:
    datos_unidos[f"SKU_STORE_mean_{window}D"] = pd.NA

    for fecha in fechas_unicas:
        # Datos del día actual
        df_dia = datos_unidos.loc[datos_unidos["DATE"] == fecha, ["SKU", "STORE_ID", "DATE", "TOTAL_SALES"]]
        df_dia_completo = rellenar_faltantes(df_dia, fecha)

        # Agregar al buffer
        buffer = pd.concat([buffer, df_dia_completo], ignore_index=True)

        # Mantener sólo los últimos window+1 días (para limitar memoria)
        if buffer["DATE"].nunique() > window+1:
            fecha_mas_vieja = buffer["DATE"].min()
            buffer = buffer[buffer["DATE"] != fecha_mas_vieja]

        # Filas originales del día actual
        df_original_dia = datos_unidos.loc[datos_unidos["DATE"] == fecha,
                                        ["SKU", "STORE_ID", "DATE", "TOTAL_SALES"]]

        # Calcular promedio con los días previos que haya 
        dias_previos = sorted(buffer["DATE"].unique())[:-1]  # todos menos el actual
        
        if len(dias_previos) > 0:
            # Tomar como máximo window días previos
            dias_a_usar = dias_previos[-window:]
            df_prev = buffer[buffer["DATE"].isin(dias_a_usar)]
            media_prev = df_prev.groupby(["SKU", "STORE_ID"], observed=False)["TOTAL_SALES"].mean().reset_index()
            media_prev["DATE"] = fecha
            media_prev.rename(columns={"TOTAL_SALES": f"SKU_STORE_mean_{window}D"}, inplace=True)

            # Actualizar directamente en el dataset original
            idx_update = datos_unidos.index[datos_unidos["DATE"] == fecha]
            merged = datos_unidos.loc[idx_update, ["SKU", "STORE_ID", "DATE"]].merge(
                media_prev, on=["SKU", "STORE_ID", "DATE"], how="left")

            # Si no se creó la columna en el merge, la creamos con NaN
            if f"SKU_STORE_mean_{window}D" not in merged.columns:
                merged[f"SKU_STORE_mean_{window}D"] = pd.NA

            datos_unidos.loc[idx_update, f"SKU_STORE_mean_{window}D"] = merged[f"SKU_STORE_mean_{window}D"].values

    datos_unidos.fillna({f"SKU_STORE_mean_{window}D":0}, inplace=True)

Ahora, haremos el promedio y desviación estándar de las ventas por subgrupo y por categoría, de manera que el modelo pueda entender mejor los cambios de ventas por épocas del año de grupos más grandes de productos.

In [ ]:
features_rolling_mean_std(datos_unidos, group="SUBGROUP", windows=[30, 90, 180])
features_rolling_mean_std(datos_unidos, group="SKU", windows=[30, 90, 180])
features_rolling_mean_std(datos_unidos, group="STORE_ID", windows=[30, 90, 180])

## Test de modelos

### LightGBM

In [ ]:
import lightgbm as lgb

In [ ]:
def walk_forward_lightgbm(df, features, target_col, date_col, categorical_cols,
                          train_days=365, step_days=30, forecast_days=7,
                          params=None):
    """
    df: DataFrame con features + target
    target_col: nombre de la columna objetivo (ej. 'TOTAL_SALES')
    date_col: columna con la fecha
    categorical_cols: lista de columnas categóricas (deben ser dtype 'category')
    train_days, step_days, forecast_days: enteros en días
    params: dict de parámetros LightGBM
    """

    df[date_col] = pd.to_datetime(df[date_col])
    df = df.sort_values(date_col)

    results = []
    min_date = df[date_col].min()
    max_date = df[date_col].max()
    start_train_end = min_date + timedelta(days=train_days)

    count = 0

    while start_train_end + timedelta(days=forecast_days) <= max_date:
        count+=1

        print(f"Walk-forward: iteracion numero {count}")

        # Train y Test
        train_data = df[df[date_col] < start_train_end]
        test_data = df[(df[date_col] >= start_train_end) &
                       (df[date_col] < start_train_end + timedelta(days=forecast_days))]

        if len(test_data) == 0:
            break

        # Creamos un validation set para early stopping
        valid_days_inner = 7
        train_end_inner = train_data["DATE"].max() - timedelta(days=valid_days_inner)

        train_inner = train_data[train_data["DATE"] <= train_end_inner]
        valid_inner = train_data[train_data["DATE"] > train_end_inner]

        X_train_inner = train_inner[features]
        y_train_inner = train_inner[target_col]
        X_valid_inner = valid_inner[features]
        y_valid_inner = valid_inner[target_col]

        # Dataset LightGBM
        lgb_train = lgb.Dataset(X_train_inner, label=y_train_inner, categorical_feature=categorical_cols)
        lgb_valid = lgb.Dataset(X_valid_inner, label=y_valid_inner, categorical_feature=categorical_cols, reference=lgb_train)

        model = lgb.train(
            params,
            lgb_train,
            valid_sets=[lgb_train, lgb_valid],
            valid_names=["train_inner", "valid_inner"]
        )

        # Predicciones
        y_test_pred = model.predict(test_data[features], num_iteration=model.best_iteration)
        y_train_pred = model.predict(X_train_inner,num_iteration=model.best_iteration)
        
        # Métricas
        r2_test = r2_score(test_data[target], y_test_pred)
        r2_train = r2_score(y_train_inner, y_train_pred)


        results.append({
            "train_end_date": start_train_end,
            "r2_train": r2_train,
            "r2_test": r2_test
        })

        start_train_end += timedelta(days=step_days)

    return pd.DataFrame(results)

In [ ]:
cols = list(datos_unidos.columns)
features = [col for col in cols if col not in ["DATE", "TOTAL_SALES",
                                               'INITIAL_TICKET_PRICE', 'BASE_PRICE', "COSTOS", 
                                               "OPENDATE", "CLOSEDATE", "QUANTITY", "STORE_SUBGROUP_DATE_ID"] ]

print(features)

In [ ]:
results_lgb = walk_forward_lightgbm(
    df=datos_unidos,
    features=features,
    target_col="TOTAL_SALES",
    date_col="DATE",
    categorical_cols=cols_categoricas,
    train_days=365,
    step_days=30,
    forecast_days=7,
    params={
        "objective": "regression",
        "metric": "rmse",
        "verbosity": 2,
        "learning_rate": 0.01,
        "num_leaves": 500,
        "max_depth": 20,
        "min_data_in_leaf": 50,
        "feature_fraction": 1,
        "bagging_fraction": 1,
        "bagging_freq": 0,
        "early_stopping_round": 20,
        "num_boost_round":1000
    }
)

In [ ]:
results_lgb.to_csv("resultados_test/resultados_lgb5.csv")

In [ ]:
results_lgb.mean()

## Deploy

### Training

In [ ]:
import lightgbm as lgb

In [ ]:
cols = list(datos_unidos.columns)
features = [col for col in cols if col not in ["DATE", "TOTAL_SALES",
                                               'INITIAL_TICKET_PRICE', 'BASE_PRICE', "COSTOS", 
                                               "OPENDATE", "CLOSEDATE", "QUANTITY", "STORE_SUBGROUP_DATE_ID"] ]

print(features)

In [ ]:
params={
        "objective": "regression",
        "metric": "rmse",
        "verbosity": 1,
        "learning_rate": 0.01,
        "num_leaves": 200,
        "max_depth": 20,
        "feature_fraction": 0.9,
        "bagging_fraction": 0.9,
        "bagging_freq": 1,
		"num_boost_round" : 100,
    }

In [ ]:
data_train = lgb.Dataset(datos_unidos[features], datos_unidos["TOTAL_SALES"], categorical_feature=cols_categoricas)

In [ ]:
model_lgb = lgb.train(params, data_train)

### Template dataframe

Primero, creamos un dataframe base con todos los productos y tiendas para los 7 días, de manera que el modelo pueda predecir las ventas de cada combinación

In [ ]:
columnas_extraidas = ['SKU', 'STORE_ID', 'REGION',
       'CITY', 'STATE', 'STORE_TYPE', 'CATEGORY', 'GROUP', 'SUBGROUP', 'GROUP_TYPE',
       'PRICE_GROUP_ID', 'BRAND', 'YEAR_OPEN', 'YEAR_CLOSE', 'MONTH_OPEN', 'MONTH_CLOSE']

In [ ]:
# Creamos un dataframe con todas las combinaciones de SKU X STORE_ID
template = datos_unidos[columnas_extraidas].drop_duplicates().reset_index(drop=True)

In [ ]:
# Agregamos los ultimos costos de los productos
ultimos_costos = (
    datos_unidos
    .groupby(["SKU", "STORE_ID"], as_index=False)
    .last()[["SKU", "STORE_ID", "COSTOS"]]
)

template = template.merge(ultimos_costos, on=["SKU", "STORE_ID"], how="left")

In [ ]:
# Quitamos las tiendas que ya cerraron
template = template[template["YEAR_CLOSE"] > 2023]

In [ ]:
# Hay 150 (numero de tiendas) . 854 (numero de sku) combinaciones
len(template)

In [ ]:
# Cada uno de los 7 dias tendra todas las combinaciones
fechas = pd.date_range(start="2024-01-01", periods=7, freq="D")
df_fechas = pd.DataFrame({"DATE": fechas})

template = (
    df_fechas.assign(key=1)
    .merge(template.assign(key=1), on="key")
    .drop(columns="key")
)

In [ ]:
# Features agregados
template["DATE"] = pd.to_datetime(template["DATE"])
template["YEAR"] = template["DATE"].dt.year
template["MONTH"] = template["DATE"].dt.month
template["DAY"] = template["DATE"].dt.day
template["DAY_OF_WEEK"] = template["DATE"].dt.day_name()
template["WEEK"] = template["DATE"].dt.isocalendar().week

In [ ]:
# Pasamos las columnas al type adecaudo
for col in cols_categoricas:
    template[col] = template[col].astype("category")

In [ ]:
# Como el modelo lo entrenaremos con 152 tiendas, pero dos de ellas cerraron y solo haremos la predicción de 150 tiendas, 
# necesitamos ajustar las categorias para que funcione correctamente lgb
type_stores = pd.api.types.CategoricalDtype(categories=datos_unidos["STORE_ID"].unique())
template["STORE_ID"] = template["STORE_ID"].astype(type_stores)

In [ ]:
template = template.sort_values(by=["DATE", "STORE_ID", "SKU"])

In [ ]:
template = features_rolling_template(df=datos_unidos, template=template, group= "SUBGROUP", windows=[30, 90, 180])
template = features_rolling_template(df=datos_unidos, template=template, group= "SKU", windows=[30, 90, 180])
template = features_rolling_template(df=datos_unidos, template=template, group= "STORE_ID", windows=[30, 90, 180])

In [ ]:
template.columns

### Optimizacion de precios

In [ ]:
price_grid = crear_price_grid(datos_unidos, n_prices=5)

In [ ]:
mejor_y_pred, mejor_sales, mejor_gain, mejor_config = optimizacion_precios(template=template, model=model_lgb, price_grid=price_grid,
                                                             features=features, n_iter=10, target="GAIN", 
                                                             save_dir="resultados_optimizacion",
                                                             file_name="mejor_config_lgb")

In [ ]:
mejor_df = pd.read_csv("resultados_optimizacion/mejor_config_lgb.csv")